<a href="https://colab.research.google.com/github/owengenge/cap-comp215/blob/main/Project_1_Plotting_NEO's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Project 1:** Plotting Potentially Hazardous Near Earth Objects

Name: Owen Genge

Date: Feb 19, 2023



In [12]:
import datetime, json, requests
from pprint import pprint   
from dataclasses import dataclass
import matplotlib.pyplot as plt


In [13]:
API_KEY = 'BQuTOmvJ8rJthGRwxNG8a02iQmcoaXEMxQ0rviXq'  

today = str(datetime.date.today())

url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-02-19&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text) 

n_results = data['element_count']
neos = data['near_earth_objects'][today]

#hazardous_neos = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]

#pprint(neos)

# api = f'https://api.nasa.gov/neo/rest/v1/neo/browse?start_date=2023-02-21&end_date=2023-02-22&api_key={API_KEY}'
# responseA = requests.request("GET", api, headers={}, data={})

# dataA = json.loads(responseA.text) 
# pprint(dataA)




In [37]:
def ast_miss_data(orbiting_body, start_date, end_date, potentially_hazardous, id):
  API_KEY = 'BQuTOmvJ8rJthGRwxNG8a02iQmcoaXEMxQ0rviXq'  
  url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?start_date={str(start_date)}&end_date={str(end_date)}&api_key={API_KEY}'
  response = requests.request("GET", url, headers={}, data={})
  data = json.loads(response.text)
  approach_data = data['close_approach_data']
  #pprint(data) 
  if data['is_potentially_hazardous_asteroid'] is potentially_hazardous:
    miss_distnaces = tuple([item['miss_distance']['kilometers'] 
                    for item in approach_data if item['orbiting_body'] == str(orbiting_body)])
    approach_dates = tuple([item['close_approach_date'] 
                    for item in approach_data if item['orbiting_body'] == str(orbiting_body)])
  else:
    print(f'NEO: {id} potentially hazardous = {not(potentially_hazardous)}')

  ast_data = (approach_dates, miss_distnaces)

  pprint(ast_data)
  # plt.scatter(ast_data)
  # plt.title(f'NEO: {id} Orbiting {orbiting_body}. Potentially Hazardous = {potentially_hazardous}')
  # plt.show

ast_miss_data(orbiting_body='Earth', start_date=today, end_date=today, potentially_hazardous=False, id='54174525')






(('1966-08-04',
  '1967-08-02',
  '1968-08-02',
  '1969-02-02',
  '1969-08-07',
  '1970-01-11',
  '1970-08-16',
  '1970-12-20',
  '1992-08-04',
  '1993-08-02',
  '1994-03-09',
  '1994-08-02',
  '1995-02-06',
  '1995-08-06',
  '1996-01-15',
  '1996-08-14',
  '1996-12-24',
  '2019-08-04',
  '2020-08-01',
  '2021-02-28',
  '2021-08-02',
  '2022-02-01',
  '2022-08-07',
  '2023-01-09',
  '2023-08-17',
  '2023-12-19',
  '2045-08-05',
  '2046-08-02',
  '2047-03-12',
  '2047-08-02',
  '2048-02-09',
  '2048-08-05',
  '2049-01-16',
  '2049-08-13',
  '2049-12-26',
  '2072-08-04',
  '2073-08-01',
  '2074-03-05',
  '2074-08-02',
  '2075-02-05',
  '2075-08-06',
  '2076-01-14',
  '2076-08-13',
  '2076-12-24',
  '2099-08-06',
  '2100-08-03',
  '2101-08-02',
  '2102-02-15',
  '2102-08-05',
  '2103-01-23',
  '2103-08-12',
  '2104-01-03',
  '2104-12-13',
  '2127-08-05',
  '2128-08-02',
  '2129-02-26',
  '2129-08-03',
  '2130-01-31',
  '2130-08-09',
  '2131-01-10',
  '2131-08-18',
  '2131-12-20',
  '2154-